In [2]:
import torch
import torch.nn as nn
from typing import Tuple, Callable
from torch.utils.data import DataLoader
import torchvision
import numpy as np
from torchsummary import summary
from tqdm import tqdm
from datetime import datetime
import os

In [14]:
def get_device() -> torch.device:
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    x = torch.ones(1, device=device)

    return device

In [15]:
def get_data(batch_size: int, resolution: Tuple[int, int]) -> Tuple[DataLoader, DataLoader]:
    transform = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize(resolution),  # 3:2 aspect ratio
            torchvision.transforms.Grayscale(num_output_channels=1),
            torchvision.transforms.ToTensor(),
        ]
    )

    data_dir = "data"
    dataset = torchvision.datasets.ImageFolder(data_dir, transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader

In [ ]:
def train(
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    loss_fn: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    loader: DataLoader,
    device: torch.device,
    epochs: int
) -> None:
    model.train()
    for epoch in range(epochs):
        progress_bar = tqdm(enumerate(iter(loader)), total=len(loader), desc=f"Epoch {epoch+1}/{epochs}")
        batch_losses = torch.empty(0).to(device)
        for batchIdx, (data, labels) in progress_bar:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = model(data)
            batch_loss = loss_fn(predictions, labels)
            batch_loss.backward()
            optimizer.step()
            batch_losses = torch.cat(batch_losses, batch_loss.item())
            progress_bar.set_postfix({'Mean batch loss': torch.mean(batch_losses)})


In [ ]:
def validate_accuracy(model: nn.Module, loader: DataLoader, device: torch.device) -> float:
    model.eval()
    is_correct = torch.empty(0).to(device)
    for i, (data, labels) in tqdm(enumerate(iter(loader))):
        data, labels = data.to(device), labels.to(device)
        with torch.no_grad():
            prediction = model(data)
        max_values, argmaxes = prediction.max(-1)
        is_correct = torch.cat((is_correct, argmaxes == labels))
    return np.mean(is_correct.cpu().numpy())

In [40]:
device = get_device()
print(f"Device: {device}")
resolution = (150, 100)
batch_size = 256
train_loader, val_loader = get_data(batch_size, resolution)

Device: cuda


In [5]:
MODEL_FILE_NAME = "rps.pt"
model = torch.load(MODEL_FILE_NAME, weights_only=False)

In [44]:
model = nn.Sequential(
    nn.Conv2d(1, 6, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(6, 16, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(11968, 128),
    nn.ReLU(),
    nn.Linear(128, 84),
    nn.ReLU(),
    nn.Linear(84, 4),
).to(device)

summary(model, (1, *resolution), batch_size, device.type)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [256, 6, 146, 96]             156
              ReLU-2          [256, 6, 146, 96]               0
         MaxPool2d-3           [256, 6, 73, 48]               0
            Conv2d-4          [256, 16, 69, 44]           2,416
              ReLU-5          [256, 16, 69, 44]               0
         MaxPool2d-6          [256, 16, 34, 22]               0
           Flatten-7               [256, 11968]               0
            Linear-8                 [256, 128]       1,532,032
              ReLU-9                 [256, 128]               0
           Linear-10                  [256, 84]          10,836
             ReLU-11                  [256, 84]               0
           Linear-12                   [256, 4]             340
Total params: 1,545,780
Trainable params: 1,545,780
Non-trainable params: 0
---------------------------

In [56]:
train(model, optimizer, loss_fn, train_loader, device, epochs=50)

Epoch 50/50: 100%|██████████| 18/18 [00:43<00:00,  2.42s/it, batch_loss=0.000547]


In [57]:
acc = validate_accuracy(model, val_loader, device)
print(f"Acc: {acc * 100:.2f}%")

Acc: 84.21%


In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
dir = os.path.join("run_artifacts", timestamp)
os.makedirs(dir, exist_ok=True)
path = os.path.join(dir, MODEL_FILE_NAME)
torch.save(model, path)

RuntimeError: Parent directory run_artifacts\20250522_071654 does not exist.